In [1]:
import pandas as pd
import numpy as np
from context_engineering_functions import create_basic_triples

context = create_basic_triples('csgo_clean', save=False)

Finished Basic Context Engineering


In [2]:
context.head()

,MatchId,de_dust2_is_available,de_inferno_is_available,de_mirage_is_available,de_nuke_is_available,de_overpass_is_available,de_train_is_available,de_vertigo_is_available,DecisionTeamId,OtherTeamId,DecisionOrder,X_Action,Y_reward
0,4,1,1,1,1,0,1,0,12,6,3,0,0
1,4,0,1,1,1,0,1,0,6,12,4,3,1
3,5,0,1,1,1,0,1,1,9,5,3,6,1
4,5,0,1,1,1,0,1,0,5,9,4,3,0
6,7,1,1,1,1,0,1,0,4,11,3,1,1


In [3]:
context.shape

(8755, 13)

In [4]:
context.reset_index(drop=True, inplace=True)

In [5]:
from bandit import Bandit

In [6]:
# do we need a train test split?

cols = [col for col in context if col.endswith('is_available')]

X = context[cols].values
A = context['X_Action'].values
Y = context['Y_reward'].values

n_arms = 7

# randomly chosen by dice roll
n = 4569
# n = np.random.choice(context.index)

bandit = Bandit(X.shape[1], n_arms, step_size=0.01)

print('Probabilities for random row: ')
print(bandit.predict_proba(X[n]))

Probabilities for random row: 
[[0.   0.25 0.25 0.25 0.   0.   0.25]]


In [7]:
n, A[n], Y[n]

(4569, 3, 1)

In [8]:
# rewards_actually_received = A[Y.astype(bool)]
# predictions = bandit.predict(X[Y.astype(bool)], deterministic=False)
# result = (predictions == rewards_actually_received).sum()
# result

In [9]:
for i in range(X.shape[0]):
    bandit.update_theta(X[i].reshape(1, -1), A[i], Y[i])

In [10]:
# for i in range(10000):
#     bandit.update_theta(X[n].reshape(1, -1), A[n], Y[n].reshape(-1, 1))
# will force action 3 - since we're observing 10k times that action 3 will reward with 1

In [11]:
print('Probabilities for random row after training: ')
print(bandit.predict_proba(X[n]))
print('Selected action: ')
print(bandit.predict(X[n]))

print('Actual action and reward for random row: ')
print(f'Action: {A[n]}, reward: {Y[n]}')

Probabilities for random row after training: 
[[0.         0.29443105 0.21140927 0.23522578 0.         0.
  0.2589339 ]]
Selected action: 
[1]
Actual action and reward for random row: 
Action: 3, reward: 1


In [12]:
# predictions = bandit.predict(X[Y.astype(bool)], deterministic=False)
# result = (predictions == rewards_actually_received).sum()
# result